### CR_1.10 - notadmin

In [8]:
from pwn import *
import binascii

# Connettiamoci al server
host = "notadmin.challs.cyberchallenge.it"
port = 9032

# Creiamo una connessione
conn = remote(host, port)

# 1. Registriamo un utente
conn.recvuntil(b"> ")
conn.sendline(b"1")
conn.recvuntil(b"Insert your username: ")
conn.sendline(b"AAAAAA")
token_line = conn.recvline().decode().strip()

# 2. Estrarre IV e Cifrato
token = token_line.split("Your login token: ")[1]
iv = bytes.fromhex(token[:32])
ciphertext = bytes.fromhex(token[32:])

# 3. Modificare l'IV per bit-flipping attack
modified_iv = bytearray(iv)
modified_iv[-2] ^= 1  # Cambia '0' in '1' nel flag is_admin=0

# 4. Creiamo il nuovo token con IV modificato
new_token = binascii.hexlify(bytes(modified_iv)).decode() + token[32:]

# 5. Login con il nuovo token
conn.recvuntil(b"> ")
conn.sendline(b"2")
conn.recvuntil(b"Insert your token: ")
conn.sendline(new_token.encode())

# 6. Leggiamo la risposta del server
response = conn.recvall().decode()
print(response)

conn.close()

[x] Opening connection to notadmin.challs.cyberchallenge.it on port 9032
[x] Opening connection to notadmin.challs.cyberchallenge.it on port 9032: Trying 5.75.221.48
[+] Opening connection to notadmin.challs.cyberchallenge.it on port 9032: Done
[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 62B
[-] Receiving all data: Failed


KeyboardInterrupt: 